In [1]:
from discussion_agents.cog.agent.react import ReActAgent

In [2]:
from langchain_community.llms.openai import OpenAI

import dotenv

dotenv.load_dotenv("../../../.env")

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

agent = ReActAgent(llm=llm)

In [7]:
q = "Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his \"unsportsmanlike conducts\" in the sport and crimes of violence outside of the ring"
out = agent.generate(observation=q)

In [8]:
print(out)

Thought 0: I need to search for a person who was once considered the best kick boxer in the world, but has been involved in controversies and crimes. 
Action 0: Search[best kick boxer in the world]
Observation 0: Jarrell Miller (born July 15, 1988)  is an American professional boxer and former kickboxer who competes in the heavyweight division He first came to prominence in 2007 when he competed for the New Jersey Tigers in the World Combat League and made it to the finals of the New York Golden Gloves tournament that same year Miller was due to challenge Anthony Joshua for the WBA (Super), IBF, WBO, and IBO heavyweight titles in 2019, but was denied a license to box after failing multiple drug tests.In 2018, Miller was ranked as the world's seventh-best heavyweight by BoxRec, and eighth by The Ring.== Early life ==Miller was born and raised in Brooklyn, New York His mother is of Belizean and Irish descent, his father is of Haitian and Dominican descent He took up Muay Thai at the age 

: 

In [2]:
if not "":
    print("A") 

A


In [3]:
("AA" if not "" else "A") + "B"

'AAB'